# Регрессия с функцией потерь Хьюбера

### Функция потерь Хьюбера

Функция потерь Хьюбера задает штраф за процедуру оценки. Хьюбер (1964) описал ее как кусочную функцию вида:

$${\displaystyle L_{\delta }(a)={\begin{cases}{\frac {1}{2}}{a^{2}}&{\text{для }}|a|\leq \delta ,\\\delta (|a|-{\frac {1}{2}}\delta ),&{\text{иначе.}}\end{cases}}}$$
Эта функция квадратична для малых значений a, и линейна для больших значений, с одинаковым значением и уклоном для различных участков двух точек где ${\displaystyle |a|=\delta }$. Переменную a часто рассматривают как остаток, т.е как разницу между наблюдаемым и предсказанным значением ${\displaystyle a=y-f(x)}$, поэтому исходное определение может быть расширено до:
$$
L_\delta(a,y)=
\begin{cases}
 \frac{1}{2}(y - a)^2,                   & |y - a| \le \delta, \\
 \delta\, |y - a| - \frac{1}{2}\delta^2, & \textrm{иначе.}
\end{cases}
$$
производная по вектору 

$$
\frac{\partial L}{\partial\omega}=\left\{\begin{array}{l}X^T(y\;-\;\omega X)\;,\;\;\;\;\;\left|y\;-\;\omega X\right|\leqslant\delta\\X^Tsign\lbrack y-\omega X\rbrack\end{array}\right.
$$

In [1]:
import pandas as pd
import numpy.linalg as la
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split

### Идея в основе градиентного спуска

Пусть имеется некая функция `f`, которая на входе принимает вектор из вещественных чисел и на выходе выдает одно-единственное вещественное число. Вот одна из таких простых функций:

```Python
def sum_of_squares(v: np.array) -> float:
    """Вычисляет сумму возведенных в квадрат элементов в v"""
    return np.dot(v, v)
```

Нередко нам необходимо максимизировать или минимизировать такие функции. Другими словами, нам нужно отыскивать вход `v`, который производит наибольшее (или наименьшее) возможное значение.

>Терминология для различных видов градиентного спуска характерна неоднород­
ностью. Подход «вычислить градиент для всего набора данных» часто называют пакетным градиентным спуском, при этом некоторые люди используют термин «стохастический градиентный спуск», ссылаясь на мини-пактную версию (среди
которых версия с одной точкой за один раз является частным случаем).

### Реализация градиентного спуска

Модель линейной регрессии для функции потерь `Huber loss`, обучаемая градиентным спуском.

Все вычисления должны быть векторизованы, циклы средствами python допускается использовать только для итераций градиентного спуска. В качестве критерия останова необходимо использовать (одновременно):

   - проверку на евклидовую норму разности весов на двух соседних итерациях (например, меньше некоторого малого числа порядка $10^{-6}$, задаваемого параметром `tolerance`);
   - достижение максимального числа итераций (например, 10000, задаваемого параметром `max_iter`).

Необходимо реализовать метод полного и стохастического (в интерпретации `mini-batch`) градиентных спусков, а также поддержать метод `momentum` при помощи параметра `alpha` (способ оценивания градиента должен задаваться при помощи параметра `gd_type`).

Чтобы проследить, что оптимизационный процесс действительно сходится, будем использовать атрибут класса `loss_history` — в нём после вызова метода `fit` должны содержаться значения функции потерь для всех итераций, начиная с первой (до совершения первого шага по антиградиенту).

Инициализировать веса можно случайным образом или нулевым вектором. Ниже приведён шаблон класса, который должен содержать код реализации модели.

In [2]:
class HuberReg(BaseEstimator):
    def __init__(self, delta=1.0, gd_type='full', tolerance=1e-4,
                 max_iter=1000, w0=None, alpha=1e-3, eta=1e-2):
        """
        gd_type: 'full' or 'stochastic'
        tolerance: for stopping gradient descent
        max_iter: maximum number of steps in gradient descent
        w0: np.array of shape (d) - init weights
        eta: learning rate
        alpha: momentum coefficient
        """
        self.delta = delta
        self.gd_type = gd_type
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.w0 = w0 
        self.alpha = alpha
        self.w = np.zeros(1)
        self.eta = eta
        self.loss_history = None
    
    def calc_loss(self, X, y):
        if la.norm(y - np.dot(X, self.w)) <= self.delta:
            return 0.5 * la.norm(y - np.dot(X, self.w))
        else:
            return self.delta * la.norm((y - np.dot(X, self.w) - 0.5 * self.delta), ord=1)
    
    def calc_gradient(self, X, y):
        step_size_0 = 0.05
        self.w = np.zeros(X.shape[1])
        if self.gd_type == 'full':
            w_mem = self.w.copy()
            h = np.zeros(X.shape[1])
            for i in range(self.max_iter):
                step_size = step_size_0 / ((i + 1) ** .5)
                if la.norm(y - np.dot(X, self.w)) <= self.delta:
                    grad = np.dot(X.T, (np.dot(X, self.w) - y)) / y.shape[0]
                    self.w -= h
                else:
                    grad = self.delta * np.dot(X.T,
                                            np.sign(np.dot(X, self.w) - y)) / y.shape[0]
                    self.w -= h
                self.loss_history.append(self.calc_loss(X, y))
                h = self.alpha * h + step_size * grad
                if np.abs(la.norm(w_mem) - la.norm(self.w)) < self.tolerance and i != 0:
                    break
            return self.w

        if self.gd_type == 'stochastic':
            batch_size = 10
            if X.shape[0] < 10:
                print('X lenght < 10')
                return None
            w_mem = self.w.copy()
            h = np.zeros(X.shape[1])
            for i in range(self.max_iter):
                sample = np.random.randint(X.shape[0], size=batch_size)                
                step_size = step_size_0 / ((i + 1) ** .5)
                if la.norm(y.iloc[sample] - np.dot(X.iloc[sample], self.w)) <= self.delta:
                    grad = np.dot(X.iloc[sample].T,
                            (np.dot(X.iloc[sample], self.w) - y.iloc[sample])) / batch_size
                    self.w -= h
                else:
                    grad = self.delta * np.dot(X.iloc[sample].T,
                     np.sign(np.dot(X.iloc[sample], self.w) - y.iloc[sample])) / batch_size
                    self.w -= h
                h = self.alpha * h + step_size * grad
                self.loss_history.append(self.calc_loss(X.iloc[sample], y.iloc[sample]))
                if la.norm(w_mem - self.w) < self.tolerance and i != 0: 
                    print("tolerance break")
        return self.w
        
    def fit(self, X, y):
        self.loss_history = []
        self.calc_gradient(X, y)
        return self
    
    def predict(self, X):
        if self.w is None:
            raise Exception('Not trained yet')
        return np.dot(X, self.w)
        
    def score(self, X, y):
        return 1 - ((y - np.dot(X, self.w)) ** 2).sum() / ((y - np.mean(y)) ** 2).sum()

### Примеры

In [3]:
data = pd.read_csv('data.csv')
data.head()

,f0,f1,f2,f3,f4,f5,f6
0,16.99,1.01,0.976270,-3.697815,0.623295,0.524760,7199.992000
1,10.34,1.66,4.303787,7.715073,0.886961,0.473862,2466.136700
2,21.01,3.50,2.055268,-6.464284,0.618826,1.657394,2969.369100
3,23.68,3.31,0.897664,1.335254,0.133461,1.234554,1040.665300
4,24.59,3.61,-1.526904,-0.196414,0.980580,3.086397,37.469975


In [4]:
# нормализация предикторов
data = data.apply(lambda column: (column - column.mean()) / column.var())

In [5]:
# разбиение на тренировочный и тестовый датасеты
X_train, X_test, y_train, y_test = train_test_split(data, data['f1'], test_size=.3)

In [8]:
%%time
hubreg = HuberReg(alpha=.8)
hubreg.fit(X_train, y_train)
print(f"R2-score: {hubreg.score(X_test, y_test)}")
print(f"W: {hubreg.w}")

R2-score: 0.9999207139481568
W: [ 7.47531897e-02  9.92153746e-01 -3.91709170e-03 -5.93706464e-03
 -8.54460756e-05  2.97173690e-04 -3.63442067e-06]
CPU times: user 2.2 s, sys: 57.3 ms, total: 2.26 s
Wall time: 2.45 s


In [9]:
%%time
hubreg = HuberReg(alpha=.8, gd_type='stochastic')
hubreg.fit(X_train, y_train)
print(f"R2-score: {hubreg.score(X_test, y_test)}")
print(f"W: {hubreg.w}")

R2-score: 0.9997968983339703
W: [ 5.97358461e-02  9.92010984e-01  2.10155648e-02 -3.86417217e-02
 -8.11426086e-05  7.31855632e-04  4.13167362e-06]
CPU times: user 4.61 s, sys: 142 ms, total: 4.76 s
Wall time: 5.35 s
